СТАТИСТИКА ПО ИСПОЛНИТЕЛЮ

In [249]:
import requests, json, urllib.parse
from statistics import mean, median

def convert_millis(millis: int) -> str:
    total_seconds = millis // 1000
    seconds = total_seconds % 60
    minutes = (total_seconds // 60) % 60
    hours = (total_seconds // (60 * 60))
    return f"{hours}H{minutes}M{seconds}S"

In [250]:
base_url = 'https://itunes.apple.com/search'

artist = 'SALUKI'

params = {
    'term':  artist,
    'limit': 15,
    'media': 'music',
    'entity': 'song',
    'attribute': 'artistTerm'
}

response = requests.get(f"{base_url}?{urllib.parse.urlencode(params)}")

data = json.loads(response.text)
data = data['results']

In [251]:
import dateutil.parser
data = map(lambda x: {**x, 'releaseDate': dateutil.parser.isoparse(x['releaseDate'])}, data)
data = list(data)


In [252]:
data = map(lambda x: {
    'artistId': x['artistId'],
    'collectionId': x['collectionId'],
    'trackId': x['trackId'],
    'artistName': x['artistName'],
    'collectionName': x['collectionCensoredName'],
    'trackName': x['trackCensoredName'],
    'collectionPrice': x['collectionPrice'],
    'trackPrice': x['trackPrice'],
    'releaseDate': x['releaseDate'],
    'collectionExplicitness': x['collectionExplicitness'],
    'trackExplicitness': x['trackExplicitness'],
    'discCount': x['discCount'],
    'discNumber': x['discNumber'],
    'trackCount': x['trackCount'],
    'trackNumber': x['trackNumber'],
    'trackTimeMillis': x['trackTimeMillis'],
    'country': x['country'],
    'currency': x['currency'],
    'primaryGenreName': x['primaryGenreName'],
    'isStreamable': x['isStreamable']
}, data)
data = list(data)


Название исполнителя

In [253]:
print(set(map(lambda x: x['artistName'], data)))

{'SALUKI', 'ANIKV & SALUKI', 'SALUKI & 104', 'Saluki= & Chu Kosaka'}


In [254]:
data = filter(lambda x: artist.lower() in x['artistName'].lower(), data)
data = list(data)

Количество песен

In [255]:
chislo = lambda x: x['trackName']
chislo_songs = len(list(map(chislo, data)))
print('Count of songs:', chislo_songs)

Count of songs: 15


Популярный жанр

In [256]:
primaryGenreName = set(list(map(lambda x: x['primaryGenreName'], data)))
print(primaryGenreName)

{'Hip-Hop/Rap', 'Indie Pop', 'J-Pop'}


Количество альбомов

In [257]:
from itertools import groupby

collection_id_extractor = lambda x: x['collectionId']

sorted_by_album = sorted(data, key=collection_id_extractor)
album_groups = {}
album_names = {}

for album_id, songs in groupby(sorted_by_album, key=collection_id_extractor):
    songs = list(songs)
    album_groups[album_id] = songs 
    album_names[album_id] = songs[0]['collectionName']

del sorted_by_album

print(list(album_names.values()))
print('count of albums:',len(set(list(map(lambda x: x['collectionId'], data)))))

['Властелин Калек', 'УЛИЦЫ, ДОМА', 'Было или нет? - Single', 'Company in Christ', 'СТЫД ИЛИ СЛАВА']
count of albums: 5


Общая длительность песен

In [258]:
time_extractor = lambda x: x['trackTimeMillis']

Duration_of_songs = sum(list(map(time_extractor, data)))
print('Duration of songs:', convert_millis(Duration_of_songs))


Duration of songs: 0H42M46S


Средняя длительность песни

In [259]:
AverageTime = int(mean(list(map(lambda x: x['trackTimeMillis'], data))))
print('Average time:',convert_millis( AverageTime))

Average time: 0H2M51S


Медиана длительности песен

In [260]:
Median_time = list(map(time_extractor, data))
print('Mediana of songs:', convert_millis(median(Median_time)))

Mediana of songs: 0H2M45S


название самой долгой песни(
длительность самой долгой песни)


название самой короткой песни(
длительность самой короткой песни)

In [261]:
time_extractor = lambda x: x['trackTimeMillis']
long_song = max(data, key=time_extractor)
short_song = min(data, key=time_extractor)
print('Longe song:', long_song['trackName'], convert_millis(long_song['trackTimeMillis']))

shortest_song = min(data, key=time_extractor)

print('Short song:', short_song['trackName'], convert_millis(short_song['trackTimeMillis']))

Longe song: KELIN (feat. OG Buda & MAYOT & Кисло-Сладкий & Bonah) 0H4M21S
Short song: OUTRO 0H1M41S


Общая стоимость песен

In [262]:
track_price = lambda x: x['trackPrice']
all_track_price= sum(list(map(track_price,data)))

print('all track price:', all_track_price,'$')


all track price: 19.349999999999994 $


Средняя стоимость песен

In [263]:
track_price = lambda x: x['trackPrice']
all_track_price= mean(list(map(track_price,data)))
print('Average price of songs',all_track_price,'$')

Average price of songs 1.29 $


Медиана стоимости песен

In [264]:
track_price = lambda x: x['trackPrice']
all_track_price= median(list(map(track_price,data)))
print('Median of track price',all_track_price,'$')

Median of track price 1.29 $


название самой дорогой песни
(стоимость самой дорогой песни)


название самой дешевой песни
(стоимость самой дешевой песни)

In [265]:
track_price = lambda x: x['trackPrice']
first=min(data, key=track_price)
print('min track price:', first['trackName'], first['trackPrice'],'$')
second=max(data, key=track_price)
print('max track price:', second['trackName'], second['trackPrice'],'$')



min track price: ДОСКИ (feat. OSA) 1.29 $
max track price: ДОСКИ (feat. OSA) 1.29 $


СТАТИСТИКА ПО АЛЬБОМУ

название альбома


In [266]:
album = list(album_groups[1561758250])
album_name = album_names[1561758250]
print(album_name)


СТЫД ИЛИ СЛАВА


Популярный жанр

In [267]:
primaryGenreName = set(list(map(lambda x: x['primaryGenreName'], album)))
print('Primary genere:',primaryGenreName)

Primary genere: {'Hip-Hop/Rap'}


Дата релиза

In [268]:
release_date_extractor = lambda x: x['releaseDate']

date = max(album, key=release_date_extractor)
print('Release date:', date['trackName'], date['releaseDate'])

Release date: ДОСКИ (feat. OSA) 2021-04-30 12:00:00+00:00


Общая длительность песен

In [269]:
timeCount = sum(list(map(lambda x: x['trackTimeMillis'], album)))
print( 'time count:',convert_millis( timeCount))


time count: 0H25M55S


Средняя длительность песен

In [270]:
AverageTime = int(mean(list(map(lambda x: x['trackTimeMillis'], album))))
print('Average time:',convert_millis(AverageTime))

Average time: 0H2M52S


Медиана длительности песен

In [271]:
Median_time = list(map(time_extractor, album))
print('Median songs:', convert_millis(median(Median_time)))

Median songs: 0H2M48S


название самой долгой песни
(длительность самой долгой песни)

название самой короткой песни
(длительность самой короткой песни)

In [272]:
time_extractor = lambda x: x['trackTimeMillis']
long_song = max(album, key=time_extractor)
short_song = min(album, key=time_extractor)
print('Longe song:', long_song['trackName'], convert_millis(long_song['trackTimeMillis']))

short_song = min(data, key=time_extractor)
print('Short song:', short_song['trackName'], convert_millis(short_song['trackTimeMillis']))

Longe song: KELIN (feat. OG Buda & MAYOT & Кисло-Сладкий & Bonah) 0H4M21S
Short song: OUTRO 0H1M41S


Общая стоимость песен

In [274]:
allTrackPrice = sum(list(map(lambda x: x['trackPrice'], album)))
print('All track price of album:',allTrackPrice,'$')

All track price of album: 11.61 $


Средняя стоимость песен

In [275]:
averageTrackPrice = mean(list(map(lambda x: x['trackPrice'], album)))
print('Average track price of album:',averageTrackPrice,'$')

Average track price of album: 1.29 $


Медиана стоимости песен

In [276]:
medianTrackPrice = median(list(map(lambda x: x['trackPrice'], album)))
print('Median track price of album:',medianTrackPrice,'$')

Median track price of album: 1.29 $


название самой дорогой песни
(стоимость самой дорогой песни)


название самой дешевой песни
(стоимость самой дешевой песни)

In [277]:
track_price = lambda x: x['trackPrice']
first=min(album, key=track_price)
print('min track price:', first['trackName'], first['trackPrice'],'$')
second=max(album, key=track_price)
print('max track price:', second['trackName'], second['trackPrice'],'$')

min track price: ДОСКИ (feat. OSA) 1.29 $
max track price: ДОСКИ (feat. OSA) 1.29 $
